This notebook contains code for heatmaps creations.

In [1]:
import pickle
import matplotlib.pyplot as plt

# %matplotlib inline
# plt.rcParams['figure.figsize'] = (15.0, 8.0)

In [2]:
# heatmap plotting functions

import scipy.cluster.hierarchy as sch
import scipy.spatial.distance as dist
import matplotlib
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt


def RedBlackGreen():
    """
    Creates red to RedtoGreenColors
    """
    cdict = {'red':   ((0.0, 0.0, 0.0),
                       (0.5, 0.0, 0.1),
                       (1.0, 1.0, 1.0)),

             'blue': ((0.0, 0.0, 0.0),
                       (1.0, 0.0, 0.0)),

             'green':  ((0.0, 0.0, 1.0),
                       (0.5, 0.1, 0.0),
                       (1.0, 0.0, 0.0))
            }
    my_cmap = matplotlib.colors.LinearSegmentedColormap('my_colormap',cdict,256)
    return my_cmap


def vals_norm(x):
    """
    Normalization for plotting
    """
    vmin=x.min()
    vmax=x.max()
    vmax = max([vmax,abs(vmin)])
    vmin = vmax*-1
    return mpl.colors.Normalize(vmin/2, vmax/2)

def get_order(my_mat):
    """
    Returns first and second dimension indencies after hirachical clustering
    """
    d_mat = dist.pdist(my_mat, 'euclidean')
    h_clust = sch.linkage(d_mat, method='complete')
    dim1_order = sch.leaves_list(h_clust)

    my_mat = my_mat.T
    d_mat = dist.pdist(my_mat, 'euclidean')
    h_clust = sch.linkage(d_mat, method='complete')
    dim2_order = sch.leaves_list(h_clust)

    return dim1_order, dim2_order


def reorder_matrix( my_mat, dim1_rnk, dim2_rnk):
    """
    Reorders matrix by given indencies
    """
    my_mat = my_mat[dim1_rnk, :]
    return my_mat[:, dim2_rnk]


def heat_map(in_mat, save_file, title='empty', xy_axis= ['Genes', 'Proteins'], cmap='gray',
             norm=None):
    """
    Creates a heatmap
    """

    fig = plt.figure()
    plot_array = np.array(in_mat)
    if not norm:
        norm=vals_norm(plot_array)
    plt.pcolor(plot_array, cmap=cmap, norm=norm)
    plt.title(title)
    plt.xlabel(xy_axis[0])
    plt.ylabel(xy_axis[1])
    plt.colorbar()
    fig.savefig(save_file, format="png", dpi=1000)


In [3]:
# Data selection
prot_rna = pickle.load(open('../../matrices/pickles/cr_z-score_raw.pkl', 'rb'))

# remove data for genes with no catrapid prediction
selection = prot_rna.any(axis=1)
prot_rna = prot_rna[selection,:]


In [4]:
prot_rna = prot_rna
# plot graphs
heat_map(prot_rna, './figures/interactions.png', cmap=RedBlackGreen())

In [4]:
# threshold check
prot_rna = prot_rna > 2.5
prot_rna = prot_rna.astype(int)
c_vals = vals_norm(prot_rna)
xnorm = mpl.colors.Normalize(0,1)

heat_map(prot_rna, './figures/interactions_gt_2_5.png', title='interactions greater than 2.5', norm=xnorm)